# ORE Version 12 - New Analytics

This dashboard demonstrates the following new analytics added in ORE version 12:
- Scenario
- Historical Simulation Var
- PNL
- PNL Explain

## Set up and load utility functions
To install ORE run **pip install open-source-risk-engine**

Dependencies:
 * pandas, numpy, lxml
 * ORE environmental variable

In [1]:
import ORE as ore

# utility script available
import sys
sys.path.append('..')
import utilities
import numpy as np
import os
import pandas as pd

from lxml import etree
def decodeXML(filename):
    return etree.tostring(etree.parse(filename)).decode('UTF-8')


## Scenario

 - Example 57
 - Returns the base scenario from a Simulation Market for a given date. 
 - The scenario is a representation of the state of the market.
 - It is a set of key/value pairs, keyed on RiskFactor:
     *  DiscountCurve/EUR/0: 1.00013288
     *  DiscountCurve/EUR/1: 1.00028576
     *  FXSpot/USDEUR/0: 0.8929
 - Requires a Simulation Parameters configuration which specifies the risk factors.
 - Scenario generation needed for Historical Simulation VaR

**ORE XML for Scenario Analytic**

In [2]:
oreXml = decodeXML("Input/Example_57/ore.xml")
print(oreXml)

<ORE>
  <Setup>
    <Parameter name="asofDate">2016-02-05</Parameter>
    <Parameter name="inputPath">Input</Parameter>
    <Parameter name="outputPath">Output</Parameter>
    <Parameter name="logFile">log.txt</Parameter>
    <Parameter name="logMask">31</Parameter>
    <Parameter name="marketDataFile">../../Input/market_20160205_flat.txt</Parameter>
    <Parameter name="fixingDataFile">../../Input/fixings_20160205.txt</Parameter>
    <Parameter name="implyTodaysFixings">N</Parameter>
    <Parameter name="curveConfigFile">../../Input/curveconfig.xml</Parameter>
    <Parameter name="conventionsFile">../../Input/conventions.xml</Parameter>
    <Parameter name="marketConfigFile">../../Input/todaysmarket.xml</Parameter>
    <Parameter name="pricingEnginesFile">../../Input/pricingengine.xml</Parameter>
    <Parameter name="observationModel">None</Parameter>
    <Parameter name="baseCurrency">EUR</Parameter>
    <Parameter name="nThreads">1</Parameter>
  </Setup>
  <Markets>
    <Parameter n

\
**Simulation XML for Scenario Analytic**

In [3]:
simMarketParamsXml = decodeXML("Input/Example_57/simulation.xml")
print(simMarketParamsXml)

<Simulation>
	<Market>
		<BaseCurrency>EUR</BaseCurrency>
		<Currencies>
			<Currency>USD</Currency>
			<Currency>EUR</Currency>
		</Currencies>
		<YieldCurves>
			<Configuration curve="">
				<Tenors>2W, 1M, 3M, 6M, 1Y, 2Y, 3Y, 5Y, 10Y, 15Y, 20Y, 30Y</Tenors>
				<Interpolation>LogLinear</Interpolation>
				<Extrapolation>FlatZero</Extrapolation>
			</Configuration>
		</YieldCurves>
		<Indices>
			<Index>USD-FedFunds</Index>
			<Index>USD-Libor-3M</Index>
			<Index>EUR-EONIA</Index>
			<Index>EUR-EURIBOR-3M</Index>
		</Indices>
		<BenchmarkCurves/>
		<FxRates>
			<CurrencyPairs/>
		</FxRates>
	</Market>
</Simulation>


\
**Setup the Scenario Analytic**

In [4]:
inputs = ore.InputParameters()

# set asof date
inputs.setAsOfDate("2016-02-05")
inputs.setResultsPath(".")
inputs.setAllFixings(True)
inputs.setEntireMarket(True)

# Read in the XML configurations
curveConfigsXml = decodeXML("../../Input/curveconfig.xml")
conventionsXml = decodeXML("../../Input/conventions.xml")
pricingEngineXml = decodeXML("../../Input/pricingengine.xml")
todaysMarketXml = decodeXML("../../Input/todaysmarket.xml")

# set configurations in InputParameters
inputs.setCurveConfigs(curveConfigsXml)
inputs.setConventions(conventionsXml)
inputs.setPricingEngine(pricingEngineXml)
inputs.setTodaysMarketParams(todaysMarketXml)

with open("../../Input/market_20160205_flat.txt") as f:
    market_data = ore.StrVector(f.read().splitlines())
    
with open("../../Input/fixings_20160205.txt") as f:
    fixings_data = ore.StrVector(f.read().splitlines())

**Set Scenario specific parameters**

In [5]:
inputs.insertAnalytic("SCENARIO")
inputs.setScenarioSimMarketParams(decodeXML("Input/Example_57/simulation.xml"))

**Run the Scenario Analytic**

In [6]:
ore_app = ore.OREApp(inputs, "log.txt", 63)
ore_app.run(market_data, fixings_data)

In [7]:
ore_app.getReportNames()

('dividends',
 'fixings',
 'marketdata',
 'runtimes',
 'scenario',
 'todaysmarketcalibration')

In [8]:
r = ore_app.getReport('scenario')
utilities.format_report(r)

,Date,Scenario,Numeraire,DiscountCurve/EUR/0,DiscountCurve/EUR/1,DiscountCurve/EUR/2,DiscountCurve/EUR/3,DiscountCurve/EUR/4,DiscountCurve/EUR/5,DiscountCurve/EUR/6,...,IndexCurve/USD-FedFunds/2,IndexCurve/USD-FedFunds/3,IndexCurve/USD-FedFunds/4,IndexCurve/USD-FedFunds/5,IndexCurve/USD-FedFunds/6,IndexCurve/USD-FedFunds/7,IndexCurve/USD-FedFunds/8,IndexCurve/USD-FedFunds/9,IndexCurve/USD-FedFunds/10,IndexCurve/USD-FedFunds/11
0,2016-02-05,1,1.0,0.999223,0.998391,0.995024,0.989988,0.980068,0.960589,0.941501,...,0.998795,0.997372,0.994342,0.987107,0.978719,0.954171,0.872913,0.780615,0.6984,0.565553



## Historical Simulation VaR

 - Example 58
 - Generates a historical simulation VaR calculation given a portfolio and a set of historical market scenarios covering
a historical observation period.
 - Load the raw historical market scenarios and use these to generate "shift" scenarios to reflect market moves for the historical period
 - Apply each shift scenario to todays simulated market sequentially, and revalue the portfolio to generate a set of PNLs
 - Calculate VaR at the quantiles provided


\
**Historical Simulation ORE XML**

In [9]:
oreXml = decodeXML("Input/Example_58/ore.xml")
print(oreXml)

<ORE>
  <Setup>
    <Parameter name="asofDate">2019-12-30</Parameter>
    <Parameter name="inputPath">Input</Parameter>
    <Parameter name="outputPath">Output</Parameter>
    <Parameter name="logFile">log.txt</Parameter>
    <Parameter name="logMask">31</Parameter>
    <Parameter name="marketDataFile">market.txt</Parameter>
    <Parameter name="fixingDataFile">fixings.txt</Parameter>
    <Parameter name="implyTodaysFixings">N</Parameter>
    <Parameter name="curveConfigFile">curveconfig.xml</Parameter>
    <Parameter name="conventionsFile">conventions.xml</Parameter>
    <Parameter name="marketConfigFile">todaysmarket.xml</Parameter>
    <Parameter name="pricingEnginesFile">pricingengine.xml</Parameter>
    <Parameter name="observationModel">None</Parameter>
    <Parameter name="baseCurrency">EUR</Parameter>
    <Parameter name="portfolioFile">portfolio.xml</Parameter>
    <Parameter name="nThreads">1</Parameter>
  </Setup>
  <Markets>
    <Parameter name="lgmcalibration">default</Par

\
**Historical Scenarios**

In [10]:
pd.read_csv('Input/Example_58/scenarios.csv')

,Date,Scenario,Numeraire,DiscountCurve/EUR/0,DiscountCurve/EUR/1,DiscountCurve/EUR/2,DiscountCurve/EUR/3,DiscountCurve/EUR/4,DiscountCurve/EUR/5,DiscountCurve/EUR/6,...,IndexCurve/USD-LIBOR-3M/3,IndexCurve/USD-LIBOR-3M/4,IndexCurve/USD-LIBOR-3M/5,IndexCurve/USD-LIBOR-3M/6,IndexCurve/USD-LIBOR-3M/7,IndexCurve/USD-LIBOR-3M/8,IndexCurve/USD-LIBOR-3M/9,IndexCurve/USD-LIBOR-3M/10,IndexCurve/USD-LIBOR-3M/11,FXSpot/USDEUR/0
0,01/09/2016,1,1,1.000133,1.000286,1.000867,1.001822,1.004012,1.008920,1.013825,...,0.995592,0.990535,0.979624,0.967804,0.941587,0.865756,0.788373,0.716837,0.598100,0.8929
1,02/09/2016,1,1,1.000133,1.000287,1.000886,1.001847,1.003999,1.008720,1.013303,...,0.995573,0.990481,0.979698,0.967993,0.941961,0.864353,0.784879,0.712156,0.591704,0.8961
2,06/09/2016,1,1,1.000134,1.000283,1.000891,1.001855,1.004134,1.009334,1.014665,...,0.995559,0.990593,0.980578,0.969741,0.945244,0.870480,0.791898,0.719285,0.598859,0.8882
3,07/09/2016,1,1,1.000133,1.000286,1.000892,1.001856,1.004147,1.009275,1.014543,...,0.995574,0.990518,0.980494,0.969410,0.944792,0.870069,0.791426,0.718388,0.597943,0.8894
4,08/09/2016,1,1,1.000132,1.000280,1.000876,1.001813,1.003982,1.008763,1.013731,...,0.995522,0.990446,0.979693,0.968098,0.942442,0.865040,0.784519,0.709840,0.586686,0.8879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812,20/12/2019,1,1,1.000175,1.000377,1.001129,1.002287,1.004640,1.009179,1.012855,...,0.990655,0.982245,0.966483,0.950550,0.917234,0.829025,0.742080,0.664310,0.538696,0.9025
813,23/12/2019,1,1,1.000175,1.000385,1.001127,1.002278,1.004640,1.009122,1.013184,...,0.990628,0.982171,0.966189,0.950037,0.916923,0.827785,0.741204,0.663156,0.537658,0.9017
814,24/12/2019,1,1,1.000174,1.000383,1.001126,1.002274,1.004593,1.009024,1.012609,...,0.990584,0.982163,0.966312,0.950552,0.917779,0.830295,0.743922,0.666849,0.541146,0.9018
815,27/12/2019,1,1,1.000173,1.000376,1.001128,1.002268,1.004597,1.009000,1.012631,...,0.990686,0.982403,0.967114,0.951541,0.919339,0.832579,0.746390,0.668883,0.543514,0.8946


\
**Set up Historical Simulation VAR calculation**

In [11]:
inputs = ore.InputParameters()

# set asof date
inputs.setAsOfDate("2019-12-30")
inputs.setResultsPath(".")
inputs.setAllFixings(True)
inputs.setEntireMarket(True)
inputs.setBaseCurrency("EUR")

curveConfigsXml = decodeXML("Input/Example_58/curveconfig.xml")
conventionsXml = decodeXML("Input/Example_58/conventions.xml")
pricingEngineXml = decodeXML("Input/Example_58/pricingengine.xml")
todaysMarketXml = decodeXML("Input/Example_58/todaysmarket.xml")
portfolioXml = decodeXML("Input/Example_58/portfolio.xml")

# set configurations in InputParameters
inputs.setCurveConfigs(curveConfigsXml)
inputs.setConventions(conventionsXml)
inputs.setPricingEngine(pricingEngineXml)
inputs.setTodaysMarketParams(todaysMarketXml)
inputs.setPortfolio(portfolioXml)

with open("Input/Example_58/market.txt") as f:
    market_data = ore.StrVector(f.read().splitlines())
    
with open("Input/Example_58/fixings.txt") as f:
    fixings_data = ore.StrVector(f.read().splitlines())

\
**Historical Simulation VaR specific parameters**

In [12]:
inputs.insertAnalytic("HISTSIM_VAR")
inputs.setHistoricalScenarioReader("Input/Example_58/scenarios.csv")
inputs.setHistVarSimMarketParamsFromFile("Input/Example_58/simulation.xml")
inputs.setBenchmarkVarPeriod("2017-01-17,2019-12-30")
inputs.setMporDays(10)
inputs.setMporCalendar("USD")
inputs.setMporOverlappingPeriods(True)
inputs.setVarQuantiles("0.01,0.05,0.95,0.99")

**Run analytic**

In [13]:
ore_app = ore.OREApp(inputs, "log.txt", 32)
ore_app.run(market_data, fixings_data)

In [14]:
ore_app.getReportNames()

('dividends',
 'fixings',
 'marketdata',
 'pricingstats',
 'runtimes',
 'todaysmarketcalibration',
 'var')

\
**VaR Report**

In [15]:
r = ore_app.getReport('var')
utilities.format_report(r)


,Portfolio,RiskClass,RiskType,Quantile_0.010000,Quantile_0.050000,Quantile_0.950000,Quantile_0.990000
0,All,All,All,-874502.874530,-629898.758251,552878.029114,846747.430447
1,All,All,DeltaGamma,-874502.874530,-629898.758251,552878.029114,846747.430447
2,All,InterestRate,All,-63274.820389,-47022.474424,75704.146136,103062.394662
3,All,InterestRate,DeltaGamma,-63274.820389,-47022.474424,75704.146136,103062.394662
4,All,FX,All,-884243.685619,-636898.503103,534980.158696,806008.620044
5,All,FX,DeltaGamma,-884243.685619,-636898.503103,534980.158696,806008.620044




## PNL

- Example 62
- Calculate the PNL between 2 dates
- Generate a SimMarket and Scenario for each date, and use them to generate 4 flavours of NPV
    * NPV(t0)
    * NPV(asof=t0; mkt=t1) - Shift the t0 SimMarket to t1's market    * 
    * NPV(t1)
    * NPV(asof=t1; mkt=t0) - Shift the t1 SimMarket to t0's market
- Calculate PeriodCashFlow - Aggregate of trade flows in the period
- We use the NPV's to calculate:
    * Theta: NPV(asof=t1; mkt=t0) - NPV(t0) + PeriodCashFLow
    * HypotheticalCleanPnL: NPV(asof=t0; mkt=t1) - NPV(t0)
    * DirtyPnL: NPV(t1) - NPV(t0)
    * CleanPnL: NPV(t1) - NPV(t0) + PeriodCashFlow 

\
**PNL ORE XML**

In [16]:
oreXml = decodeXML("Input/Example_62/ore_pnl.xml")
print(oreXml)

<ORE>
  <Setup>
    <Parameter name="asofDate">2023-01-31</Parameter>
    <Parameter name="baseCurrency">USD</Parameter>
    <Parameter name="inputPath">Input</Parameter>
    <Parameter name="outputPath">Output/Pnl</Parameter>
    <Parameter name="logFile">log.txt</Parameter>
    <Parameter name="logMask">31</Parameter>
    <Parameter name="marketDataFile">market.txt</Parameter>
    <Parameter name="fixingDataFile">fixings.txt</Parameter>
    <Parameter name="implyTodaysFixings">Y</Parameter>
    <Parameter name="fixingCutoff">2023-02-14</Parameter>
    <Parameter name="curveConfigFile">curveconfig.xml</Parameter>
    <Parameter name="conventionsFile">conventions.xml</Parameter>
    <Parameter name="marketConfigFile">todaysmarket.xml</Parameter>
    <Parameter name="pricingEnginesFile">pricingengine.xml</Parameter>
    <Parameter name="portfolioFile">portfolio.xml</Parameter>
    <Parameter name="observationModel">None</Parameter>
    <Parameter name="continueOnError">false</Parameter>

\
**Set up PNL Analytic**

In [17]:
inputs = ore.InputParameters()

# set asof date
inputs.setAsOfDate("2023-01-31")
inputs.setResultsPath(".")
inputs.setAllFixings(True)
inputs.setEntireMarket(True)
inputs.setBaseCurrency("USD")

curveConfigsXml = decodeXML("Input/Example_62/curveconfig.xml")
conventionsXml = decodeXML("Input/Example_62/conventions.xml")
pricingEngineXml = decodeXML("Input/Example_62/pricingengine.xml")
todaysMarketXml = decodeXML("Input/Example_62/todaysmarket.xml")
portfolioXml = decodeXML("Input/Example_62/portfolio.xml")

# set configurations in InputParameters
inputs.setCurveConfigs(curveConfigsXml)
inputs.setConventions(conventionsXml)
inputs.setPricingEngine(pricingEngineXml)
inputs.setTodaysMarketParams(todaysMarketXml)
inputs.setPortfolio(portfolioXml)

with open("Input/Example_62/market.txt") as f:
    market_data = ore.StrVector(f.read().splitlines())
    
with open("Input/Example_62/fixings.txt") as f:
    fixings_data = ore.StrVector(f.read().splitlines())

\
**Set PNL specific parameters**

In [18]:
inputs.insertAnalytic("PNL")
inputs.setScenarioSimMarketParamsFromFile("Input/Example_62/simulation.xml")
inputs.setMporDays(10)
inputs.setMporCalendar("USD")

\
**Run PNL Analytic**

In [19]:
ore_app = ore.OREApp(inputs, "log.txt", 125, True)
ore_app.run(market_data, fixings_data)

Requested analytics:                              PNL


In [20]:
ore_app.getReportNames()

('dividends',
 'fixings',
 'marketdata',
 'pnl',
 'pnl_cashflow',
 'pnl_npv_lagged_t0',
 'pnl_npv_lagged_t1',
 'pnl_npv_t0',
 'pnl_npv_t1',
 'pnl_npv_t1_port_t0',
 'pnl_scenario_t0',
 'pnl_scenario_t1',
 'pricingstats',
 'runtimes',
 'todaysmarketcalibration')

In [21]:
r = ore_app.getReport('pnl')
utilities.format_report(r)

,TradeId,TradeType,Maturity,MaturityTime,StartDate,EndDate,NPV(t0),NPV(asof=t0;mkt=t1),NPV(asof=t1;mkt=t0),NPV(t1;portfolio=t0),...,PeriodCashFlow,New,Matured,Terminated,Amendments,Theta,HypotheticalCleanPnL,CleanPnL,DirtyPnL,Currency
0,SwapLeg,Swap,2024-02-01,1.002508,2023-01-31,2023-02-14,47640.849246,47618.392561,47725.150117,47703.486315,...,0.000000,0.0,0.0,0.0,0.0,84.300871,-22.456685,62.637069,62.637069,USD
1,SwapLegFlow,Swap,2024-02-05,1.013437,2023-01-31,2023-02-14,97311.809777,97288.862765,47568.552161,47546.718334,...,49861.111111,0.0,0.0,0.0,0.0,117.853495,-22.947012,96.019668,-49765.091443,USD



## PNL Explain

 - Example 62 
 - Extends the PNL analytic to "explain" the PNL in terms of IR/FX/EQ etc delta/gamma/vega buckets.
 - Run a sensitivity analysis to generate sensitivities for each trade
 - Bucket each risk factor based on risk type and risk class
 - RiskTypes:

<img src="risktype.jpg" width=500 height=500 />

 - RiskClasses:
   
<img src="riskclass.jpg" width=700 height=600 />

 - Create a set of filters containing risk type / risk class pairs, from every combination.
 - Consecutively loop through each risk filter and calculate the PNL for each bucket by multiplying the sensitivities by the scenario moves for all risk factors within that filter


**PNL Explain ORE XML**

In [22]:
oreXml = decodeXML("Input/Example_62/ore_explain.xml")
print(oreXml)

<ORE>
  <Setup>
    <Parameter name="asofDate">2023-01-31</Parameter>
    <Parameter name="baseCurrency">USD</Parameter>
    <Parameter name="inputPath">Input</Parameter>
    <Parameter name="outputPath">Output/PnlExplain</Parameter>
    <Parameter name="logFile">log.txt</Parameter>
    <Parameter name="logMask">31</Parameter>
    <Parameter name="marketDataFile">market.txt</Parameter>
    <Parameter name="fixingDataFile">fixings.txt</Parameter>
    <Parameter name="implyTodaysFixings">Y</Parameter>
    <Parameter name="fixingCutoff">2023-02-14</Parameter>
    <Parameter name="curveConfigFile">curveconfig.xml</Parameter>
    <Parameter name="conventionsFile">conventions.xml</Parameter>
    <Parameter name="marketConfigFile">todaysmarket.xml</Parameter>
    <Parameter name="pricingEnginesFile">pricingengine.xml</Parameter>
    <Parameter name="portfolioFile">portfolio.xml</Parameter>
    <Parameter name="observationModel">None</Parameter>
    <Parameter name="continueOnError">false</Par

\
**Sensitivity Config**

In [23]:
sensitivityXml = decodeXML("Input/Example_62/sensitivity.xml")
print(sensitivityXml)

<SensitivityAnalysis>
	<DiscountCurves>
		<DiscountCurve ccy="EUR">
			<ShiftType>Absolute</ShiftType>
			<ShiftSize>0.0001</ShiftSize>
			<ShiftScheme>Forward</ShiftScheme>
			<ShiftTenors>2W, 1M, 3M, 6M, 1Y, 2Y, 3Y, 5Y, 10Y, 15Y, 20Y, 30Y</ShiftTenors>
			<ParConversion>
				<Instruments>DEP, DEP, DEP, DEP, OIS, OIS, OIS, OIS, OIS, OIS, OIS, OIS</Instruments>
				<SingleCurve>true</SingleCurve>
				<Conventions>
					<Convention id="DEP">EUR-ON-DEPOSIT</Convention>
					<Convention id="OIS">EUR-OIS</Convention>
				</Conventions>
			</ParConversion>
		</DiscountCurve>
		<DiscountCurve ccy="USD">
			<ShiftType>Absolute</ShiftType>
			<ShiftSize>0.0001</ShiftSize>
			<ShiftScheme>Forward</ShiftScheme>
			<ShiftTenors>2W, 1M, 3M, 6M, 1Y, 2Y, 3Y, 5Y, 10Y, 15Y, 20Y, 30Y</ShiftTenors>
			<ParConversion>
				<Instruments>DEP, DEP, DEP, DEP, OIS, OIS, OIS, OIS, OIS, OIS, OIS, OIS</Instruments>
				<SingleCurve>true</SingleCurve>
				<Conventions>
					<Convention id="DEP">USD-ON-DEPOSIT</Con

\
**Set up the PNL Explain calculation**

In [24]:
inputs = ore.InputParameters()

# set asof date
inputs.setAsOfDate("2023-01-31")
inputs.setResultsPath(".")
inputs.setAllFixings(True)
inputs.setEntireMarket(True)
inputs.setBaseCurrency("USD")

curveConfigsXml = decodeXML("Input/Example_62/curveconfig.xml")
conventionsXml = decodeXML("Input/Example_62/conventions.xml")
pricingEngineXml = decodeXML("Input/Example_62/pricingengine.xml")
todaysMarketXml = decodeXML("Input/Example_62/todaysmarket.xml")
portfolioXml = decodeXML("Input/Example_62/portfolio.xml")

# set configurations in InputParameters
inputs.setCurveConfigs(curveConfigsXml)
inputs.setConventions(conventionsXml)
inputs.setPricingEngine(pricingEngineXml)
inputs.setTodaysMarketParams(todaysMarketXml)
inputs.setPortfolio(portfolioXml)

with open("Input/Example_62/market.txt") as f:
    market_data = ore.StrVector(f.read().splitlines())
    
with open("Input/Example_62/fixings.txt") as f:
    fixings_data = ore.StrVector(f.read().splitlines())

\
**Set PNL Explain specific parameters**

In [25]:
inputs.insertAnalytic("PNL_EXPLAIN")
inputs.setScenarioSimMarketParamsFromFile("Input/Example_62/simulation.xml")
inputs.setMporDate(ore.Date(14,2,2023))
# for sensitivity analysis
simulationXml = decodeXML("Input/Example_62/simulation.xml")
inputs.setSensiSimMarketParams(simulationXml)
inputs.setSensiScenarioData(sensitivityXml)

\
**Run PNL Explain Analytic**

In [26]:
ore_app = ore.OREApp(inputs, "log.txt", 125, True)
ore_app.run(market_data, fixings_data)

Requested analytics:                              PNL_EXPLAIN


PNL Explain: Build Market                         OK
PNL Explain: Build Portfolio                      OK
Pricing: Build Market                             OK
Pricing: Build Portfolio                          OK
Risk: Sensitivity Report                          OK
Risk: PNL Explain Calculation                     

In [27]:
ore_app.getReportNames()

('dividends',
 'fixings',
 'marketdata',
 'pnl_explain',
 'pnl_scenario_t0',
 'pnl_scenario_t1',
 'pricingstats',
 'runtimes',
 'sensitivity',
 'todaysmarketcalibration')

In [28]:
r = ore_app.getReport('pnl_explain')
utilities.format_report(r)

,TradeId,TradeType,Maturity,MaturityTime,StartDate,EndDate,NPV(t0),NPV(asof=t0;mkt=t1),NPV(asof=t1;mkt=t0),NPV(t1;portfolio=t0),...,FxVega,InfDelta,InfGamma,InfVega,CreditDelta,CreditGamma,CreditVega,CommDelta,CommGamma,CommVega
0,SwapLeg,Swap,2024-02-01,1.002508,2023-01-31,2023-02-14,47640.849246,47618.392561,47725.150117,47703.486315,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SwapLegFlow,Swap,2024-02-05,1.013437,2023-01-31,2023-02-14,97311.809777,97288.862765,47568.552161,47546.718334,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
utilities.writeReport(r, [0,12,16,17,18,19,20,21])


TradeId            New                Theta              HypotheticalCleanPnL  CleanPnL           DirtyPnL           Currency           ScenarioPnl        
SwapLeg            0.0000             84.3009            -22.4567           62.6371            62.6371            USD                -22.4556           
SwapLegFlow        0.0000             117.8535           -22.9470           96.0197            -49765.0914        USD                -22.9462           




## Verify PNL Explain Delta/Gamma Calculation

**Sensitivity and Scenario Output**

In [30]:
sensitivity = ore_app.getReport('sensitivity')
sensis = utilities.format_report(sensitivity)
sensis['Factor_1'] = sensis['Factor_1'].str[:-3]
sensis

,TradeId,IsPar,Factor_1,ShiftSize_1,Factor_2,ShiftSize_2,Currency,Base NPV,Delta,Gamma
0,SwapLeg,false,DiscountCurve/USD/4,0.0001,,0.0,USD,47640.849246,-4.750831,4.738087e-04
1,SwapLeg,false,DiscountCurve/USD/5,0.0001,,0.0,USD,47640.849246,-0.026069,1.426088e-08
2,SwapLegFlow,false,DiscountCurve/USD/0,0.0001,,0.0,USD,97311.809777,-0.081902,1.346052e-07
3,SwapLegFlow,false,DiscountCurve/USD/4,0.0001,,0.0,USD,97311.809777,-4.683722,4.619964e-04
4,SwapLegFlow,false,DiscountCurve/USD/5,0.0001,,0.0,USD,97311.809777,-0.129927,3.554451e-07


In [31]:
scenario_0 = ore_app.getReport('pnl_scenario_t0')
utilities.writeReport(scenario_0, [0,19,20])


Date               DiscountCurve/USD/4  DiscountCurve/USD/5  
2023-01-31         0.9529             0.9187             


In [32]:
scenario_1 = ore_app.getReport('pnl_scenario_t1')
utilities.writeReport(scenario_1, [0,19,20])


Date               DiscountCurve/USD/4  DiscountCurve/USD/5  
2023-02-14         0.9525             0.9180             


In [33]:
t0 = utilities.format_report(scenario_0)[['DiscountCurve/USD/4','DiscountCurve/USD/5']].T
t0.columns = ['Scenario_t0']
t1 = utilities.format_report(scenario_1)[['DiscountCurve/USD/4','DiscountCurve/USD/5']].T
t1.columns = ['Scenario_t1']
df = sensis[sensis['TradeId']=='SwapLeg'][['TradeId','Factor_1','ShiftSize_1','Delta','Gamma']]
df['Time'] = df['Factor_1'].map({'DiscountCurve/USD/4': 1, 'DiscountCurve/USD/5': 2})
df = df.join(t0, on='Factor_1', how='left').join(t1, on='Factor_1', how='left')
df['ScenarioShift'] = np.log(df['Scenario_t1'] / df['Scenario_t0']) / (df['Time'] * df['ShiftSize_1'])
df['ExplainedDelta'] = df['Delta'] * df['ScenarioShift']
df['ExplainedGamma'] = 0.5 * df['Gamma'] * df['ScenarioShift'] * df['ScenarioShift']
df

,TradeId,Factor_1,ShiftSize_1,Delta,Gamma,Time,Scenario_t0,Scenario_t1,ScenarioShift,ExplainedDelta,ExplainedGamma
0,SwapLeg,DiscountCurve/USD/4,0.0001,-4.750831,4.738087e-04,1,0.952938,0.952489,-4.705121,22.353239,5.244628e-03
1,SwapLeg,DiscountCurve/USD/5,0.0001,-0.026069,1.426088e-08,2,0.918710,0.917950,-4.134170,0.107773,1.218689e-07


In [34]:
df.groupby(['TradeId'])[['ExplainedDelta', 'ExplainedGamma']].sum()

,ExplainedDelta,ExplainedGamma
TradeId,,
SwapLeg,22.461012,0.005245
